In [73]:
import os
import cv2
import math
from datetime import datetime
from ultralytics import YOLO
from collections import deque
from werkzeug.utils import secure_filename
from flask import Flask, render_template, Response, request, jsonify

In [74]:
CONF_THRESHOLD = 0.6
classNames = ["Bouledogue","Whitetip","Blacktip"]
model = YOLO("runs/detect/train16/weights/best.engine")

In [75]:
def make_pred(img, results):
    detection = False
    labels = []
    boxes_to_draw = []

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = math.ceil((box.conf[0] * 100)) / 100
            if conf > CONF_THRESHOLD:
                detection = True
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                cls = int(box.cls[0])
                class_name = classNames[cls]
                label = f'{class_name} : {conf*100:.0f}%'
                labels.append((x1, y1, label))
                boxes_to_draw.append((x1, y1, x2, y2))

    # Draw boxes and labels
    for (x1, y1, x2, y2) in boxes_to_draw:
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), 3)

    for (x1, y1, label) in labels:
        t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
        c2 = x1 + t_size[0], y1 - t_size[1] - 3
        cv2.rectangle(img, (x1, y1), c2, [255, 255, 255], -1, cv2.LINE_AA)
        cv2.putText(img, label, (x1, y1 - 2), 0, 1, [122, 0, 0], thickness=1, lineType=cv2.LINE_AA)

    return detection

In [76]:
def image_detect(image_path):
    img = cv2.imread(image_path)

    if img is None:
        return None
    
    results = model(img, verbose=False)
    make_pred(img, results)
                
    result_path = image_path.replace('uploads', 'results')
    cv2.imwrite(result_path, img)
    
    return result_path

In [77]:
def video_detection(path):
    
    tick = datetime.now()
    results = model(path, verbose=False, stream=True, half=True, conf=CONF_THRESHOLD)
    tock = datetime.now()
    print(f'Inference time: {tock - tick}')
    
    
    res = [r for r in results]
    cap = cv2.VideoCapture(path)
    
    # Config video writer
    frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    result_path = path.replace('uploads', 'results')
    
    # Config segmentation
    pre_detection_buffer_length  = int(fps * 1)
    post_detection_buffer_length = int(fps * 2)
    
    frame_idx = 0
    no_detection_count = 0
    detection_sequence_counter = 0
    in_detection_sequence = False
    
    pre_detection_buffer = deque(maxlen=pre_detection_buffer_length)
    
    tick = datetime.now()
    
    while cap.isOpened():
        success, img = cap.read()
        if not success:
            break
        
        current   = res[frame_idx]
        detection = make_pred(img, current)
            
        if detection:
            if not in_detection_sequence:
                in_detection_sequence = True
                detection_sequence_counter += 1
                video_writer = cv2.VideoWriter(f'static/results/output_{detection_sequence_counter}.mp4', fourcc, fps, (frame_width, frame_height))
                
                [video_writer.write(buffered_img) for buffered_img in pre_detection_buffer]
                pre_detection_buffer.clear()
                
            no_detection_count = 0
            video_writer.write(img)
            
        else:
            no_detection_count += 1
            if in_detection_sequence:
                if no_detection_count <= post_detection_buffer_length:
                    video_writer.write(img)
                
                else :
                    video_writer.release()
                    in_detection_sequence = False
                    pre_detection_buffer.clear()
        
        frame_idx += 1
        
    if in_detection_sequence:
        video_writer.release()
    
    tock = datetime.now()
    print(f'Drawing time: {tock - tick}')
    
    return result_path

In [78]:
app = Flask(__name__) 

app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['RESULT_FOLDER'] = 'static/results/'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif', 'mp4', 'avi', 'mov', 'PNG', 'JPG', 'JPEG', 'GIF', 'MP4', 'AVI', 'MOV'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_file():
    file = request.files['file']
    if file.filename == '':
        return jsonify(error="No selected file"), 400
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        file_extension = filename.rsplit('.', 1)[1].lower()
        if file_extension in ['mp4', 'avi', 'mov', 'MP4', 'AVI', 'MOV']:
            result_path = video_detection(filepath)
        elif file_extension in ['png', 'jpg', 'jpeg', 'JPG']:
            result_path = image_detect(filepath)
            
        if result_path:
            result_filename = os.path.basename(result_path)
            return jsonify({'filename': result_filename}), 200
    return jsonify({'error': 'Image detection failed'})
    
@app.route('/webcam')
def webcam():
    return Response(video_detection(path=0), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/gallery')
def gallery():
    images = os.listdir('static/results')
    images = [image for image in images if image.endswith(('jpg', 'jpeg', 'png', 'gif', 'JPG'))]
    return render_template('gallery.html', images=images)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Loading runs\detect\train16\weights\best.engine for TensorRT inference...
Inference time: 0:00:00.096482
Results saved to runs\detect\predict25
Drawing time: 0:00:49.935372


127.0.0.1 - - [28/May/2024 00:29:47] "POST / HTTP/1.1" 200 -
